<a href="https://colab.research.google.com/github/Mooo49/projects/blob/main/fine_tune_Bert_model_on_custom_dataset_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
covidsenti_a= pd.read_csv('/content/gg.csv')

In [3]:
!pip install neattext
# Libraries
from google.colab import drive
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from plotly import graph_objs as go
import neattext.functions as nfx # Load Text Cleaning Package
import re # for regular expressions
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Methods/Attrib
dir(nfx)
import nltk
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Remove hashtags symbol
def cleaning_URLs(data):
    data= re.sub(r'#', '', data)
    text_tokens = word_tokenize(data)
    return(data)
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(lambda data: cleaning_URLs(data))
# Transforms to lower case
def lower(text):
    low_text= text.lower()
    return low_text
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(lambda x:lower(x))
# Cleaning Text: Remove Stopwords
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_stopwords)
# Cleaning Text: Remove hyperlinks
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_urls)
# Cleaning Text: Remove @Mentions
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(lambda x: nfx.remove_userhandles(x))
# Cleaning Text: Remove Multiple WhiteSpaces
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_multiple_spaces)
# Cleaning Text: Remove Emojis
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_emojis)
# Cleaning Text: Remove Numbers
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_numbers)
# Cleaning Text: Remove Punctuations
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_puncts)
# Cleaning Text: Remove Special Characters
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(nfx.remove_special_characters)

In [5]:
from nltk.tokenize import word_tokenize
import re
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens
covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(lambda x: tokenize(x))
import nltk
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

covidsenti_a['tweet'] = covidsenti_a['tweet'].apply(lambda x: lemmatizing(x))
# Combine words into single sentence
for i in range(len(covidsenti_a['tweet'])):
    covidsenti_a['tweet'][i] = " ".join(covidsenti_a['tweet'][i])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
covidsenti_a['label'] = covidsenti_a['label'].map({'neg':1,'neu':0,'pos':2})

In [46]:
import pandas as pd

# Filter data based on label classes
class_0 = covidsenti_a[covidsenti_a['label'] == 0].sample(1000, replace=True)
class_1 = covidsenti_a[covidsenti_a['label'] == 1].sample(1000, replace=True)
class_2 = covidsenti_a[covidsenti_a['label'] == 2].sample(1000, replace=True)

# Concatenate the sampled DataFrames
balanced_df = pd.concat([class_0, class_1, class_2])

# Shuffle the DataFrame
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

# Display the first few rows of the balanced DataFrame
print(balanced_df.head())

# Check the length of the balanced DataFrame
print(len(balanced_df))

                                               tweet  label
0  dtmag bernie sander mike bloomberg democrat fa...      1
1  thing got latin class guess corona virus proba...      0
2        guard coronavirus diseasesplace order today      1
3           maybe bully fire corona virus handle rea      2
4  let ask scottyfommarketing find cure coronavir...      0
3000


In [104]:
balanced_df.tweet[3]

'maybe bully fire corona virus handle rea'

In [103]:
balanced_df['label'].value_counts()

1    1000
0    1000
2    1000
Name: label, dtype: int64

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model = model.to('cuda')

In [70]:
X = list(balanced_df["tweet"])
y = list(balanced_df["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True)

In [14]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [49]:
len(X_train),len(X_val)

(2400, 600)

In [72]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [73]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [74]:
train_dataset[5]

{'input_ids': tensor([  101,  1055, 20554, 11506, 21887, 23350,  3537,  5981, 15849,  8525,
         15388,  3147,  2330,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(1)}

In [20]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='micro')
    precision = precision_score(y_true=labels, y_pred=pred,average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [21]:
! pip install -U accelerate
! pip install -U transformers

In [22]:
import accelerate

accelerate.__version__

'0.25.0'

In [23]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.36.2', '0.25.0')

In [75]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [76]:
trainer.train()

Step,Training Loss
500,0.240100
1000,0.051200


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory output/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1315, training_loss=0.11692166709174674, metrics={'train_runtime': 128.6593, 'train_samples_per_second': 81.611, 'train_steps_per_second': 10.221, 'total_flos': 183459941658000.0, 'train_loss': 0.11692166709174674, 'epoch': 5.0})

In [77]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.41315415501594543,
 'eval_accuracy': 0.9477777777777778,
 'eval_precision': 0.9477777777777778,
 'eval_recall': 0.9477777777777778,
 'eval_f1': 0.9477777777777778,
 'eval_runtime': 1.8266,
 'eval_samples_per_second': 492.723,
 'eval_steps_per_second': 61.864,
 'epoch': 5.0}

In [78]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Remove hashtags symbol
    text = re.sub(r'#', '', text)

    # Transforms to lower case
    text = text.lower()

    # Remove hyperlinks
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove @Mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)

    # Remove multiple white spaces
    text = re.sub(r'\s+', ' ', text).strip()


    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize tokens
    lemmatized_tokens = []
    for token in filtered_tokens:
        pos = 'n'
        if wordnet.synsets(token):
            pos = wordnet.synsets(token)[0].pos()
        lemma = lemmatizer.lemmatize(token, pos=pos)
        lemmatized_tokens.append(lemma)

    # Join tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [135]:
text = "Coronavirus UK latest: Is it safe to travel? What you need to know What should I do if I think I have it?If there's https://t.co/ofozgRpLZP"
processed_text=preprocess_text(text)
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7281, -2.2971,  5.3504]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[3.0987e-04, 4.7683e-04, 9.9921e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


array([[3.0987395e-04, 4.7683349e-04, 9.9921334e-01]], dtype=float32)

In [136]:
y_pred=np.argmax(predictions)
print(y_pred)

2


In [137]:
if y_pred == 0:
  print('neutral')
elif y_pred ==1 :
  print('negative')
else :
  print('positive')


positive


In [138]:
trainer.save_model('CustomModel')